In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

from calitp_data_analysis.tables import tbls
import calitp_data_analysis.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
from rt_analysis import rt_filter_map_plot
import build_speedmaps_index

from IPython.display import display, Markdown, Latex, HTML, IFrame
import json
import base64

In [ ]:
## parameters cell
itp_id = 300

In [ ]:
analysis_date = build_speedmaps_index.ANALYSIS_DATE

In [ ]:
%%capture_parameters
human_date = analysis_date.strftime('%B %d %Y (%A)')
human_date

In [ ]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

# {organization_name}

## About These Maps:

* Each map shows bus (and rail, if applicable) speeds for {organization_name}, with a map each for the morning peak, midday, and afternoon peak periods on {human_date}.
* On the map, routes are split into segments corresponding to the distance between two stops, allowing you to focus on specific portions of the route experiencing a slowdown.
* Route segments are arrow-shaped to indicate direction of travel. Thicker segments indicate more frequent transit routes.
* State Highway Network routes are outlined in gray.
* Hover over, or click, a segment with your mouse to see the exact speed, route name, and transit service frequency in that segment. 
    * Higher-frequency routes (3+ trips per hour) are especially important, since those slowdowns correspond to more vehicles, and riders, delayed in traffic.

## Morning Peak

In [ ]:
%%capture
rt_day.set_filter(start_time='06:00', end_time='09:00')

In [ ]:
# ## avoid papermill errors when map fails to render
# m = None

In [ ]:
%%capture
_m = rt_day.segment_speed_map(how='low_speeds', no_title=True, shn=True,
                             no_render=True
                            )

In [ ]:
# m

In [ ]:
rt_day.test_gz_export()

In [ ]:
rt_day.spa_map_state['layers'][1]['properties'] = {'stroked': False}

In [ ]:
rt_day.spa_map_state['legend_url'] = 'https://storage.googleapis.com/calitp-map-tiles/legend_test.svg'

In [ ]:
rt_day.spa_map_state

In [ ]:
cbar = rt_day.speed_map_params[1]

In [ ]:
type(cbar)

In [ ]:
cbar

In [ ]:
# cbar._repr_html_()

In [ ]:
with open('./troublesome.svg', 'w') as f:
    f.write(cbar._repr_html_())

In [ ]:
# add newlines
svg = cbar._repr_html_().replace('<', '\n<')

In [ ]:
# svg from _repr_html_ missing this apperently essential header
svg_header = \
'''<?xml version="1.0" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 20010904//EN"
 "http://www.w3.org/TR/2001/REC-SVG-20010904/DTD/svg10.dtd">
 <svg version="1.0" xmlns="http://www.w3.org/2000/svg" height="50" width="500"
 viewBox="0 0 500 50">
'''

In [ ]:
# strip first svg tag from original string (already in header)
svg_strip_tag = svg.split('width="500">\n')[1]

In [ ]:
export_svg = svg_header + svg_strip_tag

In [ ]:
with open('./legend_test.svg', 'w') as f:
    f.write(export_svg)

In [ ]:
rt_day.display_spa_map()

## Variance

In [ ]:
rt_day.map_variance(no_render = True)

In [ ]:
rt_day.test_gz_export(map_type = 'variance')

In [ ]:
rt_day.spa_map_state

In [ ]:
rt_day.display_spa_map()

In [ ]:
rt_day.describe_slow_routes()

## Midday

In [ ]:
%%capture
rt_day.set_filter(start_time='10:00', end_time='14:00')

In [ ]:
## avoid papermill errors when map fails to render
m = None

In [ ]:
%%capture
_m = rt_day.segment_speed_map(how='low_speeds', no_title=True, shn=True,
                            no_render=True)

In [ ]:
# m

In [ ]:
rt_day.test_gz_export()

In [ ]:
rt_day.spa_map_state

In [ ]:
rt_day.display_spa_map()

In [ ]:
rt_day.describe_slow_routes()

## Afternoon Peak

In [ ]:
%%capture
rt_day.set_filter(start_time='15:00', end_time='19:00')

In [ ]:
## avoid papermill errors when map fails to render
m = None

In [ ]:
%%capture
_m = rt_day.segment_speed_map(how='low_speeds', no_title=True, shn=True,
                            no_render=True)

In [ ]:
# m

In [ ]:
rt_day.test_gz_export()

In [ ]:
rt_day.spa_map_state

In [ ]:
base64state = base64.urlsafe_b64encode(json.dumps(rt_day.spa_map_state).encode()).decode()
base64state

In [ ]:
f'https://leaflet-speedmaps--cal-itp-data-analyses.netlify.app/?state={base64state}'

In [ ]:
rt_day.display_spa_map()

In [ ]:
rt_day.describe_slow_routes()

In [ ]:
try:
    metrics_df = (rt_day.rt_trips
         >> group_by(_.route_short_name, _.direction_id)
         >> summarize(speed_variance = _.mean_speed_mph.var(),
                     mean_speed_mph = _.mean_speed_mph.mean(),
                     min_hour = _.median_time.min().hour,
                     max_hour = _.median_time.max().hour,
                     n = _.route_short_name.size)
         >> filter(_.min_hour <= 7, _.max_hour >= 15,
                   _.mean_speed_mph < _.mean_speed_mph.quantile(.6),
                   _.speed_variance, _.n > _.n.quantile(.4))
         >> arrange(-_.speed_variance)
         >> head(20)
        )
except Exception as e:
    pass
    # print(e)
## check that route has all-day span...

In [ ]:
def map_from_metrics(rt_day, metrics_df, which_map = 'speeds'):
    def show_title_and_map(which_map):
        try:
            am_map = rt_day.segment_speed_map(how='low_speeds', no_title = True)
            if which_map == 'speeds':
                display(Markdown(f'20th Percentile Vehicle Speeds Between Stops{rt_day.filter_formatted}'))
                display(am_map)
            elif which_map == 'variance':
                display(Markdown(f'Speed Variance Between Stops{rt_day.filter_formatted}'))
                display(rt_day.map_variance(no_title = True))
        except Exception as e:
            display(Markdown(f'Map not available for this time/route'))
            pass
        return
    try:
        route_name = metrics_df.iloc[1,0]
    except Exception as e:
        # print(e)
        return
    rt_day.set_filter(start_time='06:00', end_time='09:00',
                      route_names=[route_name])
    show_title_and_map(which_map)
    
    rt_day.set_filter(start_time='10:00', end_time='14:00',
                      route_names=[route_name])
    show_title_and_map(which_map)

    rt_day.set_filter(start_time='15:00', end_time='19:00',
                      route_names=[route_name])
    show_title_and_map(which_map)

    return

## Daily Speed Patterns

* This chart shows the distribution of median trip speeds throughout the day. It can help find times of day where transit riders experience the slowest overall speeds.

In [ ]:
try:
    rt_day.set_filter(start_time='06:00', end_time='22:00')
    display(Markdown(f'Median Trip Speed by Hour of Day{rt_day.filter_formatted}'))

    rt_day.chart_speeds(no_title=True);
except:
    display(Markdown(f'Chart not available for this agency'))
    pass

## Single-Route Detail Example (Speeds)

* These maps highlight a single route which runs throughout the day, with a low average trip speed and a high amount of variation in speeds between different trips.

In [ ]:
try:
    map_from_metrics(rt_day, metrics_df)
except Exception as e:
    # print(e)
    display(Markdown(f'Example not available for this agency'))
    pass

## Single-Route Detail Example (Variance)

* These maps highlight the same route as above, but showing _variance_ in speeds for each segment instead of 20th percentile speeds.
* Darker segments have higher variance, these are the segments where speeds are most inconsistent between individual trips.
* Segments with high variance (some fast trips and some slow trips) might be better-suited to a different set of projects than segments that experience consistently slow speeds.

In [ ]:
try:
    map_from_metrics(rt_day, metrics_df, which_map = 'variance')
except Exception as e:
    # print(e)
    display(Markdown(f'Example not available for this agency'))
    pass

# Enhanced Exports Sandbox

In [ ]:
from calitp_data.storage import get_fs

In [ ]:
fs = get_fs()

In [ ]:
test_path = "gs://calitp-map-tiles/metro_mid_TEST.geojson.gz"

In [ ]:
gdf = rt_day.detailed_map_view.copy()

In [ ]:
gdf.columns

In [ ]:
shn_gdf = rt_day.shn.copy().to_crs(shared_utils.geography_utils.WGS84)

In [ ]:
gdf = shn_gdf ## temp for shn export --> Andrew

In [ ]:
gdf = pd.concat([gdf, shn_gdf])

In [ ]:
gdf.explore()

In [ ]:
geojson_str = gdf.to_json()
geojson_bytes = geojson_str.encode('utf-8')

In [ ]:
import gzip

In [ ]:
with open('./d7_shn.geojson.gz', 'wb') as writer:
    with gzip.GzipFile(fileobj=writer, mode="w") as gz:
        gz.write(geojson_bytes)

In [ ]:
# file = fs.open(test_path, 'wb')
# gz = gzip.GzipFile(fileobj=file, mode="wb")  # use "wb" if bytes
# gz.write(geojson_bytes)
# gz.close()
# file.close()

In [ ]:
with fs.open(test_path, 'wb') as writer:
    with gzip.GzipFile(fileobj=writer, mode="w") as gz:
        gz.write(geojson_bytes)

In [ ]:
import base64
import json
state = {"name": "LA Metro Bus Speed Maps Midday",
         "url": "https://storage.googleapis.com/calitp-map-tiles/metro_mid_TEST.geojson.gz"}
base64state = base64.urlsafe_b64encode(json.dumps(state).encode()).decode()
base64state

In [ ]:
rt_day.speed_map_params[1]

In [ ]:
cmap = rt_day.speed_map_params[1]

In [ ]:
cmap.rgb_bytes_tuple(15)  # 15mph example...

In [ ]:
# base64state = base64.urlsafe_b64encode(json.dumps(rt_day.spa_map_state).encode()).decode()
# IFrame(f'https://leaflet-speedmaps--cal-itp-data-analyses.netlify.app/?state={base64state}',width=1200,height=600)